In [36]:
import pandas as pd
import numpy as np
import xgboost as xgb
import umap
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, precision_recall_curve, auc, classification_report
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Loading the Dataset

In [38]:
dataset = pd.read_csv("/content/drive/MyDrive/kaggle_datasets/creditcardfraud/creditcard.csv")
dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [39]:
dataset.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [40]:
frauds = dataset[dataset['Class'] == 1]
print(frauds.shape)   # number of fraud cases
frauds.head()         # show first few fraud rows
frauds

(492, 31)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
541,406.0,-2.312227,1.951992,-1.609851,3.997906,-0.522188,-1.426545,-2.537387,1.391657,-2.770089,...,0.517232,-0.035049,-0.465211,0.320198,0.044519,0.177840,0.261145,-0.143276,0.00,1
623,472.0,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,529.00,1
4920,4462.0,-2.303350,1.759247,-0.359745,2.330243,-0.821628,-0.075788,0.562320,-0.399147,-0.238253,...,-0.294166,-0.932391,0.172726,-0.087330,-0.156114,-0.542628,0.039566,-0.153029,239.93,1
6108,6986.0,-4.397974,1.358367,-2.592844,2.679787,-1.128131,-1.706536,-3.496197,-0.248778,-0.247768,...,0.573574,0.176968,-0.436207,-0.053502,0.252405,-0.657488,-0.827136,0.849573,59.00,1
6329,7519.0,1.234235,3.019740,-4.304597,4.732795,3.624201,-1.357746,1.713445,-0.496358,-1.282858,...,-0.379068,-0.704181,-0.656805,-1.632653,1.488901,0.566797,-0.010016,0.146793,1.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1


## Training using XGBoost

In [41]:
X = dataset.drop("Class", axis=1)
y = dataset["Class"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y
)

In [42]:
n_pos = np.sum(y_train == 1)
n_neg = np.sum(y_train == 0)
scale = n_neg / n_pos

model = xgb.XGBClassifier(
    objective="binary:logistic",
    eval_metric="aucpr",
    scale_pos_weight=scale,
    n_estimators=1000,
    learning_rate=0.025,
    max_depth=10,
    subsample=0.8,
    colsample_bytree=0.8,
    n_jobs=-1
)

model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    verbose=25
)

y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))

precision, recall, _ = precision_recall_curve(y_test, y_proba)
print("PR-AUC:", auc(recall, precision))

print("\nClassification Report:\n", classification_report(y_test, y_pred))

[0]	validation_0-aucpr:0.64798
[25]	validation_0-aucpr:0.77973
[50]	validation_0-aucpr:0.77440
[75]	validation_0-aucpr:0.78561
[100]	validation_0-aucpr:0.78194
[125]	validation_0-aucpr:0.79718
[150]	validation_0-aucpr:0.81151
[175]	validation_0-aucpr:0.81342
[200]	validation_0-aucpr:0.82842
[225]	validation_0-aucpr:0.85907
[250]	validation_0-aucpr:0.86423
[275]	validation_0-aucpr:0.87005
[300]	validation_0-aucpr:0.87367
[325]	validation_0-aucpr:0.87837
[350]	validation_0-aucpr:0.87972
[375]	validation_0-aucpr:0.88145
[400]	validation_0-aucpr:0.88169
[425]	validation_0-aucpr:0.88204
[450]	validation_0-aucpr:0.88247
[475]	validation_0-aucpr:0.88278
[500]	validation_0-aucpr:0.88389
[525]	validation_0-aucpr:0.88406
[550]	validation_0-aucpr:0.88435
[575]	validation_0-aucpr:0.88453
[600]	validation_0-aucpr:0.88450
[625]	validation_0-aucpr:0.88486
[650]	validation_0-aucpr:0.88510
[675]	validation_0-aucpr:0.88511
[700]	validation_0-aucpr:0.88538
[725]	validation_0-aucpr:0.88534
[750]	validatio

## Plotting Normal vs Fraud and Prediction Results

In [43]:
y_pred_series = pd.Series(y_pred, index=X_test.index)
y_proba_series = pd.Series(y_proba, index=X_test.index)

fraud_idx = y_test[y_test == 1].index
normal_idx = y_test[y_test == 0].sample(1000).index
subset_idx = fraud_idx.union(normal_idx)

X_subset = X_test.loc[subset_idx]
y_subset = y_test.loc[subset_idx]
y_pred_subset = y_pred_series.loc[subset_idx]
y_proba_subset = y_proba_series.loc[subset_idx]

reducer = umap.UMAP()
X_2d = reducer.fit_transform(X_subset)

plot_df = pd.DataFrame({
    "UMAP1": X_2d[:,0],
    "UMAP2": X_2d[:,1],
    "TrueClass": y_subset.values,
    "PredictedClass": y_pred_subset.values,
    "FraudProbability": y_proba_subset.values
})

fig_truth = px.scatter(
    plot_df, x="UMAP1", y="UMAP2",
    color=plot_df["TrueClass"].map({0:"Normal", 1:"Fraud"}),
    title="True Fraud vs Normal (UMAP Projection)",
    opacity=0.7
)
fig_truth.show()

fig_pred = px.scatter(
    plot_df, x="UMAP1", y="UMAP2",
    color=plot_df["PredictedClass"].map({0:"Normal", 1:"Fraud"}),
    size="FraudProbability",
    title="XGBoost Predictions on Test Set (UMAP Projection)",
    opacity=0.7
)
fig_pred.show()


In [44]:
conditions = []
for true, pred in zip(y_subset, y_pred_subset):
    if true == 0 and pred == 0:
        conditions.append("True Negative")
    elif true == 1 and pred == 1:
        conditions.append("True Positive")
    elif true == 0 and pred == 1:
        conditions.append("False Positive")
    elif true == 1 and pred == 0:
        conditions.append("False Negative")

plot_df["ErrorType"] = conditions

fig_errors = px.scatter(
    plot_df, x="UMAP1", y="UMAP2",
    color="ErrorType",
    title="Error Analysis: Fraud Detection Outcomes (UMAP Projection)",
    symbol="ErrorType",
    opacity=0.7,
)

fig_errors.show()

## Conclusion

For this project I decided to go with XGBoost as it is particularly good for tabular datasets like this one where each row and each column represents one thing with no inconsistencies. Unlike deep learning or neuroevolution methods, which can be better for images, text or inconsistent datasets with varying data types, ranges and points; XGBoost can handle smaller feature sets efficiently and gives excellent performance on row/column based imbalanced data. It also trains much faster and is easier to tune compared to more complex approaches.

The results were strong: the model reached over 99 percent overall accuracy, though this is easy to reach as the dataset is heavily skewed towards non-fraudulent activity, but with fraud detection performance at around 94 percent precision and 84 percent recall with a 89 percent F1 score which is the main target. This means it was able to correctly identify most fraudulent transactions while keeping false positives relatively low.

Through dimensionality reduction with UMAP, PCA and t-SNE, we also saw that fraud cases cluster in small, distinct regions rather than being dispersed equally throughout the dataset, which helps explain why XGBoost was effective.

Finally, the error analysis showed that the few missed frauds tend to be cases that are hard to separate from normal transactions, suggesting that these would be challenging for any model.

Overall, the project shows that XGBoost is both a practical and reliable choice for credit card fraud detection, balancing accuracy, interpretability and training efficiency.